# Pycosat implementation of Algo #1

In [1]:
import pycosat

In [2]:
# Solver for Numberlink
class NumberLinkSolver:
    pass

In [ ]:
# choose variables
# we need a number per variable
# ok so each (x, y) is a vertex
# x_{vertex, index of path, position}
# position <= n**2 (a path could be all the vertices)
def x(self, vertex, index, position):
    """
    vertex being None means the phantom vertex
    """
    n = self.n  # number of rows and columns
    num_paths = self.num_paths  # number of paths

    if vertex is not None:
        x, y = vertex
        vertex_num = x * n + y + 1  # +1 because 0 is the phantom vertex
    else:
        vertex_num = 0

    # index is based on how many paths we have
    ans = vertex_num * num_paths * (n**2 + 1) + index * (n**2 + 1) + position
    # +1 because we start from 1
    return ans + 1


NumberLinkSolver.x = x


def inverse_x(self, var):
    n = self.n
    num_paths = self.num_paths

    var -= 1

    rest, position = divmod(var, n**2 + 1)
    vertex_num, index = divmod(rest, num_paths)
    if vertex_num == 0:
        vertex = None
    else:
        vertex_num -= 1
        x = vertex_num // n
        y = vertex_num % n
        vertex = (x, y)

    return vertex, index, position


NumberLinkSolver.inverse_x = inverse_x

In [90]:
# HELPER METHODS
def at_least_one(literals):
    return [literals]


def at_most_one(literals):
    return [[-l1, -l2] for l1 in literals for l2 in literals if l1 < l2]


def exactly_one(literals):
    return at_least_one(literals) + at_most_one(literals)

In [91]:
def each_vertex_in_a_single_path(self):
    n = self.n
    num_paths = self.num_paths

    clauses = []
    for vertex in self.vertices:
        literals = [
            self.x(vertex, index, position)
            for index in range(num_paths)
            for position in range(n**2 + 1)
        ]
        clauses += exactly_one(literals)
    return clauses


NumberLinkSolver.each_vertex_in_a_single_path = each_vertex_in_a_single_path

In [92]:
def each_position_in_a_single_vertex(self):
    n = self.n
    num_paths = self.num_paths

    clauses = []
    for index in range(num_paths):
        for position in range(n**2 + 1):
            literals = [self.x(vertex, index, position) for vertex in self.vertices] + [
                self.x(None, index, position)
            ]
            clauses += exactly_one(literals)
    return clauses


NumberLinkSolver.each_position_in_a_single_vertex = each_position_in_a_single_vertex

In [93]:
def path_finished(self):
    n = self.n
    num_paths = self.num_paths

    clauses = []
    for index in range(num_paths):
        for position in range(n**2):
            clauses.append(
                [-self.x(None, index, position), self.x(None, index, position + 1)]
            )
    return clauses


NumberLinkSolver.path_finished = path_finished

In [100]:
def consecutive_vertices_along_path(self):
    # def get_neighbors(vertex):
    #     x, y = vertex
    #     neighbors = []
    #     for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
    #         nx, ny = x + dx, y + dy
    #         if 0 <= nx < n and 0 <= ny < n:
    #             neighbors.append((nx, ny))
    #     return neighbors
    def is_neighbor(vertex1, vertex2):
        x1, y1 = vertex1
        x2, y2 = vertex2
        return abs(x1 - x2) + abs(y1 - y2) == 1

    n = self.n
    num_paths = self.num_paths
    x = self.x

    # enforce that the next vertex is a neighbor of the current vertex

    clauses = []
    for index in range(num_paths):
        for position in range(n**2):
            for vertex1 in self.vertices:
                for vertex2 in self.vertices:
                    if not is_neighbor(vertex1, vertex2):
                        clauses.append([-x(vertex1, index, position), -x(vertex2, index, position + 1)])
                # # need at least one neighbor
                # literals = [x(vertex2, index, position + 1) for vertex2 in get_neighbors(vertex1)]
                # # if this vertex is in the path, then the next vertex must be a neighbor
                # literals.append(-x(vertex1, index, position))
                # # OR it's a phantom vertex
                # literals.append(x(None, index, position + 1))
    return clauses


NumberLinkSolver.consecutive_vertices_along_path = consecutive_vertices_along_path

In [101]:
def source_and_sink(self):
    n = self.n
    num_paths = self.num_paths
    pairs = self.pairs
    x = self.x

    clauses = []
    for index in range(num_paths):
        clauses.append([x(pairs[index][0], index, 0)])
        for position in range(n**2):
            clauses.append(
                [-x(pairs[index][1], index, position), x(None, index, position + 1)]
            )
        literals = [x(pairs[index][1], index, position) for position in range(n**2)]
        clauses += exactly_one(literals)
    return clauses


NumberLinkSolver.source_and_sink = source_and_sink

In [102]:
from collections import defaultdict


def __init__(self, puzzle):
    # puzzle is a list of lists of integers where 0 is empty, and any other number is a vertex
    n = len(puzzle)
    pair_matches = defaultdict(list)
    for x in range(n):
        for y in range(n):
            vertex = puzzle[x][y]
            if vertex != 0:
                pair_matches[vertex].append((x, y))
    pairs = list(pair_matches.values())

    self.n = n
    self.pairs = pairs
    self.num_paths = len(pairs)
    self.vertices = [(x, y) for x in range(n) for y in range(n)]


NumberLinkSolver.__init__ = __init__

In [103]:
def solve(self):
    clauses = []
    clauses += self.each_vertex_in_a_single_path()
    clauses += self.each_position_in_a_single_vertex()
    clauses += self.path_finished()
    clauses += self.consecutive_vertices_along_path()
    clauses += self.source_and_sink()
    print(len(clauses)) 

    literals = pycosat.solve(clauses)

    if literals == "UNSAT":
        return None

    n = self.n
    solution = [[0 for _ in range(n)] for _ in range(n)]
    for literal in literals:
        if literal > 0:
            vertex, index, _ = self.inverse_x(literal)
            if vertex is None:
                continue
            x, y = vertex
            if solution[x][y] != 0:
                raise ValueError("Multiple paths in the same cell")
            solution[x][y] = index + 1

    return solution


NumberLinkSolver.solve = solve

In [104]:
puzzle = [[1, 0, 1], [2, 0, 2], [3, 0, 3]]

solver = NumberLinkSolver(puzzle)
print(solver.solve())  # [[1, 1, 1], [2, 2, 2], [3, 3, 3]]

7011
[[1, 1, 1], [2, 2, 2], [3, 3, 3]]


In [105]:
import json

with open("puzzles.json", "r") as f:
    puzzles = json.load(f)
    puzzles.sort(key=lambda x: x["dimension"])
puzzle_sizes = [puzzle["dimension"] for puzzle in puzzles]
puzzle_sizes

[4, 5, 6, 7, 8, 9, 10, 11]

In [106]:
from tqdm import tqdm

In [88]:
puzzle

[[0, 0, 0, 1], [0, 4, 3, 0], [0, 0, 4, 0], [0, 1, 3, 0]]

In [109]:
solver = NumberLinkSolver(puzzle)
solution = solver.solve()
for row in solution:
    print(row)

37849
[1, 1, 1, 1]
[1, 2, 3, 3]
[3, 2, 2, 3]
[3, 3, 3, 3]
